In [1]:
import pickle
import gensim

In [2]:
f = open("../Dataset/BOW.txt", "r")

for line in f:
    list_of_words = line.split(',')
f.close()

index_to_words = {}
for i in range(len(list_of_words)):
    index_to_words[i+1] = list_of_words[i]

In [3]:
f = open("../Dataset/dataset.pkl", "rb")
msdid_to_lyrics =  pickle.load(f)
f.close()

msd_ids = list(msdid_to_lyrics.keys())

In [4]:
processed_docs = []

for msd_id in msd_ids:
    lyrics = msdid_to_lyrics[msd_id][-1]

    word_lyrics = []
    for c in lyrics:
        collon_index = c.find(':')
        ind = int(c[:collon_index])
        freq = int(c[collon_index+1:])
        for i in range(freq):
            word_lyrics.append(index_to_words[ind])
    processed_docs.append(word_lyrics)

In [5]:
print(len(processed_docs))

237662


In [6]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [7]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 a
1 am
2 and
3 are
4 arrang
5 be
6 been
7 captur
8 damn
9 de
10 devast


In [8]:
dictionary.filter_extremes(no_below=15, no_above=0.5)

In [9]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)